####  load dataset 

In [31]:
from  pyspark.sql import SparkSession
from pyspark.sql import *
from pyspark.sql.functions import countDistinct
from pyspark.sql import functions as f
# from pyspark.sql.functions import isNull
from pyspark.sql.functions import col, udf ,desc
import pyspark.sql.types as t
from pyspark.sql.types import *

In [4]:
spark=SparkSession.builder.appName("mycsv_file").getOrCreate()

* read csv file 

In [5]:
df= spark .read.csv("ecommerce_customer_data_large.csv" ,header=True)


* show our data set 

In [6]:
df.show()
df.count()

+-----------+-------------------+----------------+-------------+--------+---------------------+--------------+------------+-------+--------------+---+------+-----+
|Customer ID|      Purchase Date|Product Category|Product Price|Quantity|Total Purchase Amount|Payment Method|Customer Age|Returns| Customer Name|Age|Gender|Churn|
+-----------+-------------------+----------------+-------------+--------+---------------------+--------------+------------+-------+--------------+---+------+-----+
|      44605|2023-05-03 21:30:02|            Home|          177|       1|                 2427|        PayPal|          31|    1.0|   John Rivera| 31|Female|    0|
|      44605|2021-05-16 13:57:44|     Electronics|          174|       3|                 2448|        PayPal|          31|    1.0|   John Rivera| 31|Female|    0|
|      44605|2020-07-13 06:16:57|           Books|          413|       1|                 2345|   Credit Card|          31|    1.0|   John Rivera| 31|Female|    0|
|      44605|202

250000

In [7]:
df.printSchema()

root
 |-- Customer ID: string (nullable = true)
 |-- Purchase Date: string (nullable = true)
 |-- Product Category: string (nullable = true)
 |-- Product Price: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Total Purchase Amount: string (nullable = true)
 |-- Payment Method: string (nullable = true)
 |-- Customer Age: string (nullable = true)
 |-- Returns: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Churn: string (nullable = true)



#### EDA

* let's detect null vaules 

In [8]:
df.describe().show()

+-------+-----------------+-------------------+----------------+----------------+-----------------+---------------------+--------------+-----------------+------------------+---------------+-----------------+------+------------------+
|summary|      Customer ID|      Purchase Date|Product Category|   Product Price|         Quantity|Total Purchase Amount|Payment Method|     Customer Age|           Returns|  Customer Name|              Age|Gender|             Churn|
+-------+-----------------+-------------------+----------------+----------------+-----------------+---------------------+--------------+-----------------+------------------+---------------+-----------------+------+------------------+
|  count|           250000|             250000|          250000|          250000|           250000|               250000|        250000|           250000|            202618|         250000|           250000|250000|            250000|
|   mean|     25017.632092|               NULL|            NULL|

In [9]:
df= df.withColumnRenamed("Product Category","Product_Category")
df= df.withColumnRenamed("Total Purchase Amount","Total_Purchase_Amount")
df= df.withColumnRenamed("Total Purchase Amount","Total_Purchase_Amount")
df= df.withColumnRenamed("Customer Age","Customer_Age")
df= df.withColumnRenamed("Customer Name","Customer_Name")
df= df.withColumnRenamed("Product Price","Product_price")
df.show()

+-----------+-------------------+----------------+-------------+--------+---------------------+--------------+------------+-------+--------------+---+------+-----+
|Customer ID|      Purchase Date|Product_Category|Product_price|Quantity|Total_Purchase_Amount|Payment Method|Customer_Age|Returns| Customer_Name|Age|Gender|Churn|
+-----------+-------------------+----------------+-------------+--------+---------------------+--------------+------------+-------+--------------+---+------+-----+
|      44605|2023-05-03 21:30:02|            Home|          177|       1|                 2427|        PayPal|          31|    1.0|   John Rivera| 31|Female|    0|
|      44605|2021-05-16 13:57:44|     Electronics|          174|       3|                 2448|        PayPal|          31|    1.0|   John Rivera| 31|Female|    0|
|      44605|2020-07-13 06:16:57|           Books|          413|       1|                 2345|   Credit Card|          31|    1.0|   John Rivera| 31|Female|    0|
|      44605|202

#### checking null values in a specific column and handling missing values

* *RETURNS* column has 47K null values

In [10]:
# Count null values in a specific column
null_count = df.filter(df.Returns.isNull()).count()
null_count

47382

* Count null values in a other columns

In [11]:

null_count = df.filter(df.Customer_Age.isNull()).count()
null_count

0

In [12]:
# Count null values in a specific column
null_count = df.filter(df.Customer_Name.isNull()).count()
null_count

0

> drop two columns "unused columns" (Returns , Churn)

* we have dropped two columns and clean all nul values

In [13]:
df=df.drop("Returns")
df=df.drop("Churn")

* show table after drop two columns

In [14]:
df.show()

+-----------+-------------------+----------------+-------------+--------+---------------------+--------------+------------+--------------+---+------+
|Customer ID|      Purchase Date|Product_Category|Product_price|Quantity|Total_Purchase_Amount|Payment Method|Customer_Age| Customer_Name|Age|Gender|
+-----------+-------------------+----------------+-------------+--------+---------------------+--------------+------------+--------------+---+------+
|      44605|2023-05-03 21:30:02|            Home|          177|       1|                 2427|        PayPal|          31|   John Rivera| 31|Female|
|      44605|2021-05-16 13:57:44|     Electronics|          174|       3|                 2448|        PayPal|          31|   John Rivera| 31|Female|
|      44605|2020-07-13 06:16:57|           Books|          413|       1|                 2345|   Credit Card|          31|   John Rivera| 31|Female|
|      44605|2023-01-17 13:14:36|     Electronics|          396|       3|                  937|     

#### Groubing and Sorting 

> sorting customer ID ascendingly 

In [15]:
df=df.sort("Customer ID")
df.show()

+-----------+-------------------+----------------+-------------+--------+---------------------+--------------+------------+--------------+---+------+
|Customer ID|      Purchase Date|Product_Category|Product_price|Quantity|Total_Purchase_Amount|Payment Method|Customer_Age| Customer_Name|Age|Gender|
+-----------+-------------------+----------------+-------------+--------+---------------------+--------------+------------+--------------+---+------+
|          1|2021-04-08 18:33:34|           Books|          456|       5|                 5148|   Credit Card|          67| Dominic Cline| 67|Female|
|          1|2022-11-29 06:48:25|     Electronics|          459|       5|                  757|   Credit Card|          67| Dominic Cline| 67|Female|
|          1|2020-03-04 10:26:02|        Clothing|          205|       5|                  385|          Cash|          67| Dominic Cline| 67|Female|
|         10|2022-07-06 12:30:15|           Books|          113|       5|                 4841|     

#### how many customer and how many category has been purchased for each customer

* most category used is home categories

In [16]:
amount_product=df.groupBy("Product_Category").agg(countDistinct("Customer ID").alias("amount product purchase")).show()
amount_product

+----------------+-----------------------+
|Product_Category|amount product purchase|
+----------------+-----------------------+
|            Home|                  35752|
|     Electronics|                  35713|
|        Clothing|                  35654|
|           Books|                  35504|
+----------------+-----------------------+



* removing dublicates for each customer and category

* show number of categories have been purchased by customers

In [17]:
df_dup=df.select(["Customer ID" , "Product_Category"]).distinct()
s=df_dup.sort("Customer ID")
s.show()

+-----------+----------------+
|Customer ID|Product_Category|
+-----------+----------------+
|          1|           Books|
|          1|        Clothing|
|          1|     Electronics|
|         10|           Books|
|         10|     Electronics|
|        100|        Clothing|
|        100|           Books|
|        100|            Home|
|        100|     Electronics|
|       1000|     Electronics|
|       1000|        Clothing|
|       1000|            Home|
|      10000|            Home|
|      10000|        Clothing|
|      10000|           Books|
|      10001|           Books|
|      10001|     Electronics|
|      10003|            Home|
|      10003|     Electronics|
|      10003|           Books|
+-----------+----------------+
only showing top 20 rows



* show number of customers in data set 

In [18]:
df.select("Customer ID").distinct().count()

49661


#### What is the total price for each customer's purchase

* let us create column that returns total price of purchased product

In [19]:
df2=df.withColumn("Total_Purchace_Price", col("Product_Price")*col("Total_Purchase_Amount")).show()
df2

+-----------+-------------------+----------------+-------------+--------+---------------------+--------------+------------+--------------+---+------+--------------------+
|Customer ID|      Purchase Date|Product_Category|Product_price|Quantity|Total_Purchase_Amount|Payment Method|Customer_Age| Customer_Name|Age|Gender|Total_Purchace_Price|
+-----------+-------------------+----------------+-------------+--------+---------------------+--------------+------------+--------------+---+------+--------------------+
|          1|2021-04-08 18:33:34|           Books|          456|       5|                 5148|   Credit Card|          67| Dominic Cline| 67|Female|           2347488.0|
|          1|2022-11-29 06:48:25|     Electronics|          459|       5|                  757|   Credit Card|          67| Dominic Cline| 67|Female|            347463.0|
|          1|2020-03-04 10:26:02|        Clothing|          205|       5|                  385|          Cash|          67| Dominic Cline| 67|Fem

#### what most payment method used

In [20]:
pay_amount = df.groupBy("Payment Method").agg(countDistinct("Customer ID").alias("Payment Method amount")).show()
pay_amount

+--------------+---------------------+
|Payment Method|Payment Method amount|
+--------------+---------------------+
|   Credit Card|                40527|
|        PayPal|                40382|
|          Cash|                40548|
+--------------+---------------------+



* show each customer's payment method used

In [21]:
cust_pay_amount=df.groupBy("Customer ID").agg(countDistinct("Payment Method").alias("customer payment amount"))
cust_pay_amount.show()

+-----------+-----------------------+
|Customer ID|customer payment amount|
+-----------+-----------------------+
|       9583|                      3|
|      42370|                      2|
|      28334|                      2|
|      30966|                      3|
|      35374|                      3|
|       9030|                      3|
|      48402|                      3|
|       4032|                      2|
|      29865|                      3|
|      20219|                      3|
|      42688|                      3|
|       5925|                      3|
|       9009|                      3|
|      48147|                      2|
|      39457|                      3|
|      28503|                      2|
|      49290|                      3|
|      43085|                      3|
|      40740|                      3|
|      35670|                      3|
+-----------+-----------------------+
only showing top 20 rows



#### join new column 

> let' join Customer Payment amount to our dataset 

In [22]:
df2 = df.join(cust_pay_amount ,on="Customer ID" , how = "inner")
df2.show()

+-----------+-------------------+----------------+-------------+--------+---------------------+--------------+------------+--------------+---+------+-----------------------+
|Customer ID|      Purchase Date|Product_Category|Product_price|Quantity|Total_Purchase_Amount|Payment Method|Customer_Age| Customer_Name|Age|Gender|customer payment amount|
+-----------+-------------------+----------------+-------------+--------+---------------------+--------------+------------+--------------+---+------+-----------------------+
|      44605|2023-05-03 21:30:02|            Home|          177|       1|                 2427|        PayPal|          31|   John Rivera| 31|Female|                      3|
|      44605|2021-05-16 13:57:44|     Electronics|          174|       3|                 2448|        PayPal|          31|   John Rivera| 31|Female|                      3|
|      44605|2020-07-13 06:16:57|           Books|          413|       1|                 2345|   Credit Card|          31|   John

* classify Gender number

In [23]:
gender_amount = df2.groupBy("Gender").agg(countDistinct("Customer ID").alias("GenderclassNum")).show()
gender_amount

+------+--------------+
|Gender|GenderclassNum|
+------+--------------+
|Female|         24714|
|  Male|         24947|
+------+--------------+



In [24]:
gender_amount = df2.groupBy("Customer ID").agg(countDistinct("Purchase Date").alias("DistinctPurchaseDateCount"))
gender_amount.show()

+-----------+-------------------------+
|Customer ID|DistinctPurchaseDateCount|
+-----------+-------------------------+
|      39457|                        9|
|       9583|                        8|
|      21783|                        2|
|      27317|                       10|
|      35374|                        7|
|      40740|                        8|
|      28135|                        4|
|      42776|                        9|
|      28334|                        5|
|       1512|                        5|
|      16504|                        8|
|      35670|                        7|
|      32558|                        6|
|      29865|                        9|
|       9030|                       10|
|      49290|                        7|
|      17686|                        4|
|       9009|                        6|
|      43085|                        5|
|      20219|                        4|
+-----------+-------------------------+
only showing top 20 rows



* join DistinctPurchaseDateCount into df2 data set 

In [25]:
df2 = df2.join(gender_amount ,on="Customer ID" , how = "inner")
df2.show()

+-----------+-------------------+----------------+-------------+--------+---------------------+--------------+------------+--------------+---+------+-----------------------+-------------------------+
|Customer ID|      Purchase Date|Product_Category|Product_price|Quantity|Total_Purchase_Amount|Payment Method|Customer_Age| Customer_Name|Age|Gender|customer payment amount|DistinctPurchaseDateCount|
+-----------+-------------------+----------------+-------------+--------+---------------------+--------------+------------+--------------+---+------+-----------------------+-------------------------+
|      44605|2023-05-03 21:30:02|            Home|          177|       1|                 2427|        PayPal|          31|   John Rivera| 31|Female|                      3|                        5|
|      44605|2021-05-16 13:57:44|     Electronics|          174|       3|                 2448|        PayPal|          31|   John Rivera| 31|Female|                      3|                        5|


In [26]:
df2.printSchema()

root
 |-- Customer ID: string (nullable = true)
 |-- Purchase Date: string (nullable = true)
 |-- Product_Category: string (nullable = true)
 |-- Product_price: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Total_Purchase_Amount: string (nullable = true)
 |-- Payment Method: string (nullable = true)
 |-- Customer_Age: string (nullable = true)
 |-- Customer_Name: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- customer payment amount: long (nullable = false)
 |-- DistinctPurchaseDateCount: long (nullable = false)



#### Rank With *Payment Method* and order by *customer ID*

In [27]:
from pyspark.sql.functions import row_number ,rank ,dense_rank
from pyspark.sql.window import Window

In [28]:
df2.show()

+-----------+-------------------+----------------+-------------+--------+---------------------+--------------+------------+--------------+---+------+-----------------------+-------------------------+
|Customer ID|      Purchase Date|Product_Category|Product_price|Quantity|Total_Purchase_Amount|Payment Method|Customer_Age| Customer_Name|Age|Gender|customer payment amount|DistinctPurchaseDateCount|
+-----------+-------------------+----------------+-------------+--------+---------------------+--------------+------------+--------------+---+------+-----------------------+-------------------------+
|      44605|2023-05-03 21:30:02|            Home|          177|       1|                 2427|        PayPal|          31|   John Rivera| 31|Female|                      3|                        5|
|      44605|2021-05-16 13:57:44|     Electronics|          174|       3|                 2448|        PayPal|          31|   John Rivera| 31|Female|                      3|                        5|


In [29]:
widow = Window.partitionBy("Payment Method").orderBy("Customer ID")
df2.withColumn("row number", row_number().over(widow)).show()


+-----------+-------------------+----------------+-------------+--------+---------------------+--------------+------------+--------------------+---+------+-----------------------+-------------------------+----------+
|Customer ID|      Purchase Date|Product_Category|Product_price|Quantity|Total_Purchase_Amount|Payment Method|Customer_Age|       Customer_Name|Age|Gender|customer payment amount|DistinctPurchaseDateCount|row number|
+-----------+-------------------+----------------+-------------+--------+---------------------+--------------+------------+--------------------+---+------+-----------------------+-------------------------+----------+
|          1|2021-04-08 18:33:34|           Books|          456|       5|                 5148|   Credit Card|          67|       Dominic Cline| 67|Female|                      2|                        3|         1|
|          1|2022-11-29 06:48:25|     Electronics|          459|       5|                  757|   Credit Card|          67|       Do